# Tensorflow MNIST dataset

## Depth = 3

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

/home/harshal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/harshal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/harshal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/harshal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

## Outline the Model

In [2]:
input_size = 784
output_size = 10
hidden_layer_size = 50

tf.reset_default_graph() #clear the variables in the memory left from previous run

inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

#layer 1
weights_1 = tf.get_variable("weights_1", shape=[input_size,hidden_layer_size])
biases_1 = tf.get_variable("biases_1", shape=[hidden_layer_size])

outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

#layer 2
weights_2 = tf.get_variable("weights_2", shape = [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", shape=[hidden_layer_size])

outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

#layer 3
weights_3 = tf.get_variable("weights_3", shape = [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", shape = [output_size])

outputs = tf.matmul(outputs_2,weights_3) + biases_3 #activate function is applied here
                                                    # that can be encorporated in loss function

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## We use softmax function cross entropy function with logit.
### It applies softmax function on the output and calculates cross entopy simulateously.

### We use this function when dealing with small numbers because it is numerically stable function.

### logit function here gives unscaled probabilities, softmax scales the probabilities 

In [3]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)

mean_loss = tf.reduce_mean(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## Optimize funfction

In [4]:
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

### Calculating accuracy of out model by comparing outputs and targets, we check for each observation in target matches with our output.

### We check the columns for each with max probability in the output is equal to target.

### argmax returns the index(horizontal) of the column in which we find the highest argument.

### tf.equal returns a boolean.

### out_equal_target has an array of 1s and 0s so accuracy is mean of that, as 1s means model get the correct output and 0 means otherwise.

In [5]:
out_equal_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1)) 

In [6]:
accuracy = tf.reduce_mean(tf.cast(out_equal_target, tf.float32)) # cast is like astype to change the dtype

In [7]:
sess = tf.InteractiveSession()

initialiser = tf.global_variables_initializer()

sess.run(initialiser)

In [10]:
batch_size = 100

batch_number = mnist.train.num_examples // batch_size

max_epochs = 15

prev_validation_loss = 9999999.

for epoch in range(max_epochs):
    curr_loss = 0
    for batch in range(batch_number):
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        _, batch_loss = sess.run([optimize, mean_loss], feed_dict={inputs:input_batch,targets: target_batch})
        curr_loss += batch_loss
    
    curr_loss /=batch_number
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation.num_examples)
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                                                   feed_dict = {inputs:input_batch, targets: target_batch})
    print(f"epoch = {epoch}")
    print("curr_loss = {0:.3f}".format(curr_loss))
    print("validation_loss = {0:.3f}".format(validation_loss))
    print("validation_accuracy = {0:.3f}".format(validation_accuracy))
    print("-"*25)
    
    if validation_loss > prev_validation_loss:
        break
    prev_validation_loss = validation_loss
print("End of training.")


epoch = 0
curr_loss = 0.169
validation_loss = 0.141
validation_accuracy = 0.960
-------------------------
epoch = 1
curr_loss = 0.128
validation_loss = 0.124
validation_accuracy = 0.964
-------------------------
epoch = 2
curr_loss = 0.104
validation_loss = 0.109
validation_accuracy = 0.969
-------------------------
epoch = 3
curr_loss = 0.088
validation_loss = 0.102
validation_accuracy = 0.969
-------------------------
epoch = 4
curr_loss = 0.076
validation_loss = 0.096
validation_accuracy = 0.971
-------------------------
epoch = 5
curr_loss = 0.067
validation_loss = 0.092
validation_accuracy = 0.972
-------------------------
epoch = 6
curr_loss = 0.059
validation_loss = 0.091
validation_accuracy = 0.973
-------------------------
epoch = 7
curr_loss = 0.052
validation_loss = 0.101
validation_accuracy = 0.972
-------------------------
End of training.


In [11]:
input_batch, target_batch = mnist.test.next_batch(batch_size)
test_accuracy = sess.run([accuracy], feed_dict={inputs: input_batch, targets:target_batch})
test_accuracy_percent = (test_accuracy[0]*100).round(2)
print(f"Test accuracy = {test_accuracy_percent}%")

Test accuracy = 96.0%
